In [1]:
import numpy as np
import matplotlib.pyplot as plt
import spacy
from spacy import displacy
import nltk
import pandas as pd

In [2]:
nlp = spacy.load('en_core_web_sm')

In [3]:
# nlp.pipeline,
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

Regular Expression

In [ ]:
import re
txt = 'the phone number is 458-566-1202 ,and our email is example@gmail.com.'

In [ ]:
patrn = r'\d\d\d-\d\d\d-\d\d\d\d' #for searching phone number
re.search(patrn,txt)

In [ ]:
email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')
email_text = "Contact support@example.com for assistance or info@company.co.uk for more details."
emails = email_pattern.findall(email_text)
emails

In [ ]:
#removing numbers and punctuation from the sentence
' ' .join(re.findall(r'[^!?.,\d]+','there is 3 numbers, in  1 lines out of 10 sentence!right?'))

In [ ]:
pattern = re.compile(r'm.n')
result = pattern.findall('men can do everything if the man has patient')
result

In [ ]:
re.findall(r'.at','this is a flat,there is a cat wearing a')

In [4]:
doc = nlp(u'it is a $10 doller gift card from Google.you can get it from "www.fgm.com" website! like seriously!!!')

In [5]:
for t in doc:
  print(t,t.pos_,t.lemma_)

it PRON it
is AUX be
a DET a
$ SYM $
10 NUM 10
doller NOUN doller
gift NOUN gift
card NOUN card
from ADP from
Google.you PROPN Google.you
can AUX can
get VERB get
it PRON it
from ADP from
" PUNCT "
www.fgm.com PROPN www.fgm.com
" PUNCT "
website NOUN website
! PUNCT !
like VERB like
seriously ADV seriously
! PUNCT !
! PUNCT !
! PUNCT !


In [6]:
displacy.render(doc,style = 'dep',jupyter = True,options={'distance':80})

### Stemming

In [7]:
pd.DataFrame(nlp.Defaults.stop_words)

,0
0,neither
1,somewhere
2,did
3,or
4,front
...,...
321,whatever
322,of
323,on
324,his


In [8]:
df = pd.read_csv('/content/smsspamcollection.tsv',sep='\t')
df.head(10)

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2
5,spam,FreeMsg Hey there darling it's been 3 week's n...,147,8
6,ham,Even my brother is not like to speak with me. ...,77,2
7,ham,As per your request 'Melle Melle (Oru Minnamin...,160,6
8,spam,WINNER!! As a valued network customer you have...,157,6
9,spam,Had your mobile 11 months or more? U R entitle...,154,2


In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X = df['message']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [163]:
vectorizer = CountVectorizer()

X_vect = vectorizer.fit_transform(X_train)

# feature names (words)
feature_names = vectorizer.get_feature_names_out()

#dense matrix
dense_matrix = X_vect.toarray()

print("Feature Names (Words):", pd.DataFrame(feature_names))
print("Feature Matrix:\n", dense_matrix)

Feature Names (Words):              0
0           00
1          000
2        0009f
3          007
4           03
...        ...
33150     zurg
33151      zus
33152    zwick
33153  zwigoff
33154    zycie

[33155 rows x 1 columns]
Feature Matrix:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]


In [13]:
tfidf = TfidfVectorizer()


# Fit and transform the text data
X_tfidf = tfidf.fit_transform(X_train)

# Get feature names (words)
feature_names_tfidf = tfidf.get_feature_names_out()

# Convert to dense matrix (for demonstration purposes)
dense_matrix_tfidf = X_tfidf.toarray()

print("Feature Names (Words):", feature_names_tfidf)
print("TF-IDF Feature Matrix:\n", dense_matrix_tfidf)


Feature Names (Words): ['00' '000' '000pes' ... 'zoom' 'zouk' 'èn']
TF-IDF Feature Matrix:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [14]:
movie_df = pd.read_csv('/content/moviereviews.tsv',sep='\t')
movie_df

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...
...,...,...
1995,pos,"i like movies with albert brooks , and i reall..."
1996,pos,it might surprise some to know that joel and e...
1997,pos,the verdict : spine-chilling drama from horror...
1998,pos,i want to correct what i wrote in a former ret...


In [15]:
movie_df.isnull().sum()

label      0
review    35
dtype: int64

In [16]:
movie_df.dropna(inplace = True)

In [17]:
X = movie_df['review']
y = movie_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

# Naïve Bayes:
text_clf_nb = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', MultinomialNB()),
])

# Linear SVC:
text_clf_lsvc = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

In [166]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'tfidf__max_features': [1000, 5000],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__C': [0.1, 1.0, 10.0],
    'clf__penalty': ['l1', 'l2'],
}

# Grid Search
grid_search = GridSearchCV(text_clf_lsvc, param_grid, cv=5)
grid_search.fit(X_train, y_train)

#best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

#best model for prediction
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py", line 274, in fit
    self.coef_, self.intercept_, n_iter_ = _fit_liblinear(
  File "/usr/loca

In [168]:
print(best_model)
print(predictions)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=5000, ngram_range=(1, 2))),
                ('clf', LinearSVC())])
['pos' 'neg' 'neg' 'neg' 'neg' 'pos' 'neg' 'pos' 'neg' 'pos' 'neg' 'pos'
 'neg' 'pos' 'neg' 'pos' 'neg' 'neg' 'neg' 'neg' 'neg' 'neg' 'pos' 'neg'
 'neg' 'neg' 'pos' 'neg' 'neg' 'pos' 'neg' 'pos' 'neg' 'neg' 'neg' 'neg'
 'pos' 'pos' 'neg' 'neg' 'neg' 'pos' 'neg' 'neg' 'neg' 'neg' 'neg' 'pos'
 'pos' 'pos' 'neg' 'neg' 'neg' 'pos' 'neg' 'neg' 'neg' 'neg' 'neg' 'neg'
 'neg' 'pos' 'neg' 'pos' 'pos' 'neg' 'neg' 'neg' 'neg' 'pos' 'neg' 'neg'
 'neg' 'pos' 'pos' 'neg' 'neg' 'pos' 'neg' 'pos' 'neg' 'neg' 'pos' 'neg'
 'pos' 'neg' 'neg' 'pos' 'pos' 'neg' 'pos' 'neg' 'neg' 'pos' 'neg' 'neg'
 'pos' 'neg' 'neg' 'pos' 'neg' 'neg' 'neg' 'pos' 'neg' 'neg' 'pos' 'neg'
 'pos' 'pos' 'pos' 'pos' 'neg' 'neg' 'neg' 'neg' 'pos' 'neg' 'pos' 'neg'
 'neg' 'neg' 'neg' 'neg' 'pos' 'pos' 'neg' 'pos' 'neg' 'pos' 'pos' 'pos'
 'pos' 'pos' 'neg' 'pos' 'pos' 'neg' 'pos' 'pos' 'neg' 'ne

In [169]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[274  48]
 [ 51 276]]


In [170]:
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

         neg       0.84      0.85      0.85       322
         pos       0.85      0.84      0.85       327

    accuracy                           0.85       649
   macro avg       0.85      0.85      0.85       649
weighted avg       0.85      0.85      0.85       649



In [171]:
print(metrics.accuracy_score(y_test,predictions))

0.847457627118644


In [24]:
text_clf_lsvc.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [25]:
predictions = text_clf_lsvc.predict(X_test)

In [26]:
print(metrics.confusion_matrix(y_test,predictions))

[[281  41]
 [ 56 271]]


In [27]:
print(metrics.accuracy_score(y_test,predictions))

0.8505392912172574


In [28]:
stopwords = ['a', 'about', 'an', 'and', 'are', 'as', 'at', 'be', 'been', 'but', 'by', 'can', \
             'even', 'ever', 'for', 'from', 'get', 'had', 'has', 'have', 'he', 'her', 'hers', 'his', \
             'how', 'i', 'if', 'in', 'into', 'is', 'it', 'its', 'just', 'me', 'my', 'of', 'on', 'or', \
             'see', 'seen', 'she', 'so', 'than', 'that', 'the', 'their', 'there', 'they', 'this', \
             'to', 'was', 'we', 'were', 'what', 'when', 'which', 'who', 'will', 'with', 'you']

### adding stop words to imporve calssification

In [172]:
text_clf_lsvc2 = Pipeline([('tfidf', TfidfVectorizer(stop_words=stopwords)),
                     ('clf', LinearSVC()),
])
grid_search2 = GridSearchCV(text_clf_lsvc2, param_grid, cv=5)
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py", line 274, in fit
    self.coef_, self.intercept_, n_iter_ = _fit_liblinear(
  File "/usr/loca

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words=['a',
                                                                    'about',
                                                                    'an', 'and',
                                                                    'are', 'as',
                                                                    'at', 'be',
                                                                    'been',
                                                                    'but', 'by',
                                                                    'can',
                                                                    'even',
                                                                    'ever',
                                                                    'for',
                                                                    'from',
                                                                    'get',
                                                                    'had',
                                                                    'has',
                                                                    'have',
                                                                    'he', 'her',
                                                                    'hers',
                                                                    'his',
                                                                    'how', 'i',
                                                                    'if', 'in',
                                                                    'into',
                                                                    'is', ...])),
                                       ('clf', LinearSVC())]),
             param_grid={'clf__C': [0.1, 1.0, 10.0],
                         'clf__penalty': ['l1', 'l2'],
                         'tfidf__max_features': [1000, 5000],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]})

In [174]:
best_model2 = grid_search2.best_estimator_
predictions = best_model2.predict(X_test)
print(metrics.confusion_matrix(y_test,predictions))

[[269  53]
 [ 50 277]]


In [175]:
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

         neg       0.84      0.84      0.84       322
         pos       0.84      0.85      0.84       327

    accuracy                           0.84       649
   macro avg       0.84      0.84      0.84       649
weighted avg       0.84      0.84      0.84       649



In [176]:
print(metrics.accuracy_score(y_test,predictions))

0.8412942989214176


In [33]:
myreview1 = "A movie I really wanted to love was terrible. \
I'm sure the producers had the best intentions, but the execution was lacking."

In [34]:
myreview2 = "as a thriller movie fan i was looking for a movie that is full of thrill and this movie \
did not dissapoint me. though i thought it won't meet my expection but i was wrong."

In [35]:
myreview3 = "Marvel's 'Avengers: Endgame' is a breathtaking cinematic masterpiece that beautifully wraps up over a decade of storytelling. \
 The movie impressively weaves together multiple story arcs, delivering an emotional and action-packed experience. \
  With stunning visual effects, compelling character development, and an epic conclusion,\
   it's a must-watch for every fan of the Marvel Cinematic Universe."

In [178]:
print(best_model.predict([myreview1]))
print(best_model2.predict([myreview2]))
print(best_model.predict([myreview3]))

['neg']
['neg']
['pos']


In [37]:
### Using lemmatization for performence improvement

In [38]:
import spacy

# Load Spacy's English model
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Lemmatization and stopwords
def spacy_lemmatize(text):
    return ' '.join([token.lemma_ for token in nlp(text) if not token.is_stop])

# Applying Lemmatization and custom stopwords
X_train_lemmatized = [spacy_lemmatize(text) for text in X_train]
X_test_lemmatized = [spacy_lemmatize(text) for text in X_test]

# Updating the model pipeline with Lemmatized text
text_clf_lsvc3 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LinearSVC())
])

text_clf_lsvc3.fit(X_train_lemmatized, y_train)

predictions = text_clf_lsvc3.predict(X_test_lemmatized)

print(metrics.confusion_matrix(y_test, predictions))
print(metrics.classification_report(y_test, predictions))
print(metrics.accuracy_score(y_test, predictions))


[[264  58]
 [ 55 272]]
              precision    recall  f1-score   support

         neg       0.83      0.82      0.82       322
         pos       0.82      0.83      0.83       327

    accuracy                           0.83       649
   macro avg       0.83      0.83      0.83       649
weighted avg       0.83      0.83      0.83       649

0.8258859784283513


In [39]:
print(text_clf_lsvc3.predict([myreview1]))
print(text_clf_lsvc3.predict([myreview2])) #it should be posative
print(text_clf_lsvc3.predict([myreview3]))

['neg']
['neg']
['pos']


### semantic and sentiment analysis

In [40]:
!python -m spacy download en_core_web_lg

2024-01-04 08:48:12.375430: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-04 08:48:12.375495: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-04 08:48:12.376698: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-04 08:48:13.383248: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [41]:
nlp2 = spacy.load('en_core_web_lg',disable=['parser','tagger','ner'])

In [42]:
# nlp2(u'love').vector

In [43]:
tokens = nlp2(u'lion cat pet')
for i in tokens:
  for j in tokens:
    print(i.text,j.text,i.similarity(j))

lion lion 1.0
lion cat 0.3854507803916931
lion pet 0.20031584799289703
cat lion 0.3854507803916931
cat cat 1.0
cat pet 0.732966423034668
pet lion 0.20031584799289703
pet cat 0.732966423034668
pet pet 1.0


/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [44]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [45]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [46]:
sid = SentimentIntensityAnalyzer()

In [47]:
a = 'this is a fucking good movie'
sid.polarity_scores(a)

{'neg': 0.0, 'neu': 0.556, 'pos': 0.444, 'compound': 0.4927}

In [48]:
sid.polarity_scores(myreview2)

{'neg': 0.119, 'neu': 0.748, 'pos': 0.132, 'compound': -0.3716}

In [49]:
sid.polarity_scores(movie_df.iloc[202]['review'])

{'neg': 0.098, 'neu': 0.752, 'pos': 0.15, 'compound': 0.9892}

In [50]:
movie_df['scores'] = movie_df['review'].apply(lambda r : sid.polarity_scores(r))
movie_df.head(10)

,label,review,scores
0,neg,how do films like mouse hunt get into theatres...,"{'neg': 0.121, 'neu': 0.778, 'pos': 0.101, 'co..."
1,neg,some talented actresses are blessed with a dem...,"{'neg': 0.12, 'neu': 0.775, 'pos': 0.105, 'com..."
2,pos,this has been an extraordinary year for austra...,"{'neg': 0.068, 'neu': 0.781, 'pos': 0.15, 'com..."
3,pos,according to hollywood movies made in last few...,"{'neg': 0.071, 'neu': 0.782, 'pos': 0.147, 'co..."
4,neg,my first press screening of 1998 and already i...,"{'neg': 0.091, 'neu': 0.817, 'pos': 0.093, 'co..."
5,neg,"to put it bluntly , ed wood would have been pr...","{'neg': 0.123, 'neu': 0.821, 'pos': 0.056, 'co..."
6,neg,"synopsis : melissa , a mentally-disturbed woma...","{'neg': 0.087, 'neu': 0.742, 'pos': 0.17, 'com..."
7,neg,tim robbins and martin lawernce team up in thi...,"{'neg': 0.118, 'neu': 0.709, 'pos': 0.172, 'co..."
8,neg,"in "" gia "" , angelina jolie plays the titular ...","{'neg': 0.082, 'neu': 0.862, 'pos': 0.056, 'co..."
9,neg,"in 1990 , the surprise success an unheralded l...","{'neg': 0.145, 'neu': 0.728, 'pos': 0.127, 'co..."


In [51]:
movie_df['compound'] = movie_df['scores'].apply(lambda d : d['compound'])
movie_df.head(10)

,label,review,scores,compound
0,neg,how do films like mouse hunt get into theatres...,"{'neg': 0.121, 'neu': 0.778, 'pos': 0.101, 'co...",-0.9125
1,neg,some talented actresses are blessed with a dem...,"{'neg': 0.12, 'neu': 0.775, 'pos': 0.105, 'com...",-0.8618
2,pos,this has been an extraordinary year for austra...,"{'neg': 0.068, 'neu': 0.781, 'pos': 0.15, 'com...",0.9951
3,pos,according to hollywood movies made in last few...,"{'neg': 0.071, 'neu': 0.782, 'pos': 0.147, 'co...",0.9972
4,neg,my first press screening of 1998 and already i...,"{'neg': 0.091, 'neu': 0.817, 'pos': 0.093, 'co...",-0.2484
5,neg,"to put it bluntly , ed wood would have been pr...","{'neg': 0.123, 'neu': 0.821, 'pos': 0.056, 'co...",-0.9855
6,neg,"synopsis : melissa , a mentally-disturbed woma...","{'neg': 0.087, 'neu': 0.742, 'pos': 0.17, 'com...",0.9871
7,neg,tim robbins and martin lawernce team up in thi...,"{'neg': 0.118, 'neu': 0.709, 'pos': 0.172, 'co...",0.9829
8,neg,"in "" gia "" , angelina jolie plays the titular ...","{'neg': 0.082, 'neu': 0.862, 'pos': 0.056, 'co...",-0.8278
9,neg,"in 1990 , the surprise success an unheralded l...","{'neg': 0.145, 'neu': 0.728, 'pos': 0.127, 'co...",-0.9147


In [52]:
movie_df['compound_score'] = movie_df['compound'].apply(lambda s : 'pos' if s > 0 else 'neg' )
movie_df.head(10)

,label,review,scores,compound,compound_score
0,neg,how do films like mouse hunt get into theatres...,"{'neg': 0.121, 'neu': 0.778, 'pos': 0.101, 'co...",-0.9125,neg
1,neg,some talented actresses are blessed with a dem...,"{'neg': 0.12, 'neu': 0.775, 'pos': 0.105, 'com...",-0.8618,neg
2,pos,this has been an extraordinary year for austra...,"{'neg': 0.068, 'neu': 0.781, 'pos': 0.15, 'com...",0.9951,pos
3,pos,according to hollywood movies made in last few...,"{'neg': 0.071, 'neu': 0.782, 'pos': 0.147, 'co...",0.9972,pos
4,neg,my first press screening of 1998 and already i...,"{'neg': 0.091, 'neu': 0.817, 'pos': 0.093, 'co...",-0.2484,neg
5,neg,"to put it bluntly , ed wood would have been pr...","{'neg': 0.123, 'neu': 0.821, 'pos': 0.056, 'co...",-0.9855,neg
6,neg,"synopsis : melissa , a mentally-disturbed woma...","{'neg': 0.087, 'neu': 0.742, 'pos': 0.17, 'com...",0.9871,pos
7,neg,tim robbins and martin lawernce team up in thi...,"{'neg': 0.118, 'neu': 0.709, 'pos': 0.172, 'co...",0.9829,pos
8,neg,"in "" gia "" , angelina jolie plays the titular ...","{'neg': 0.082, 'neu': 0.862, 'pos': 0.056, 'co...",-0.8278,neg
9,neg,"in 1990 , the surprise success an unheralded l...","{'neg': 0.145, 'neu': 0.728, 'pos': 0.127, 'co...",-0.9147,neg


###  Latent Dirichlet Allocation (LDA)

In [53]:
len(movie_df['review'])

1965

In [54]:
cv2 = CountVectorizer(max_df = 0.95,min_df = 2,stop_words='english')
dtm = cv2.fit_transform(movie_df['review'])
dtm

<1965x23298 sparse matrix of type '<class 'numpy.int64'>'
	with 455692 stored elements in Compressed Sparse Row format>

In [55]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=8)

In [56]:
lda.fit(dtm)

LatentDirichletAllocation(n_components=8)

### Grab the topics

In [57]:
lda.components_

array([[ 1.43681582, 49.76818237,  0.12512775, ...,  0.12502707,
        17.82663099,  0.12508218],
       [ 0.125     ,  2.8461892 ,  8.76238646, ...,  3.03461257,
         1.35491206,  0.125     ],
       [ 0.12517256, 19.31933299,  0.12506855, ...,  0.125     ,
         0.192624  ,  0.125     ],
       ...,
       [ 0.125     ,  3.28617274,  0.12519487, ...,  0.21525185,
         0.12503538,  2.12471203],
       [ 0.125     ,  1.46102683,  0.1259176 , ...,  0.125     ,
         0.1253701 ,  0.12502755],
       [ 4.02296508, 26.6236003 ,  0.12501565, ...,  0.1251085 ,
         0.12502713,  0.12504692]])

In [58]:
single_topic = lda.components_[0]
toptenwords = single_topic.argsort()[-10:]

In [59]:
for i in toptenwords:
  print(cv2.get_feature_names_out()[i])

characters
way
story
time
character
good
just
like
movie
film


In [60]:
for i,topic in enumerate(lda.components_):
   print(f'top 10 words for topic {i}')
   print([cv2.get_feature_names_out()[i] for i in topic.argsort()[-10:]])

top 10 words for topic 0
['characters', 'way', 'story', 'time', 'character', 'good', 'just', 'like', 'movie', 'film']
top 10 words for topic 1
['way', 'story', 'good', 'plot', 'just', 'time', 'characters', 'like', 'movie', 'film']
top 10 words for topic 2
['bad', 'ship', 'effects', 'time', 'story', 'just', 'alien', 'like', 'film', 'movie']
top 10 words for topic 3
['story', 'does', 'man', 'good', 'character', 'love', 'like', 'movie', 'life', 'film']
top 10 words for topic 4
['new', 'funny', 'little', 'make', 'really', 'good', 'just', 'like', 'film', 'movie']
top 10 words for topic 5
['young', 'make', 'time', 'man', 'new', 'story', 'life', 'movie', 'film', 'joe']
top 10 words for topic 6
['comedy', 'characters', 'bad', 'films', 'time', 'good', 'just', 'like', 'movie', 'film']
top 10 words for topic 7
['little', 'characters', 'character', 'good', 'just', 'story', 'time', 'like', 'movie', 'film']


In [61]:
topic_result = lda.transform(dtm)
topic_result

array([[1.51830767e-01, 6.69514001e-04, 6.69536867e-04, ...,
        6.69257079e-04, 6.69350771e-04, 8.44152674e-01],
       [9.51119128e-01, 4.21561873e-04, 4.21510171e-04, ...,
        4.21518287e-04, 4.63514930e-02, 4.21614958e-04],
       [6.21221743e-01, 7.80945437e-02, 4.18654854e-04, ...,
        4.18867205e-04, 4.18711540e-04, 2.28127324e-01],
       ...,
       [8.11160714e-01, 3.26029912e-04, 1.90766850e-02, ...,
        3.26041854e-04, 8.78354498e-02, 3.26073768e-04],
       [4.36130259e-04, 4.36173113e-04, 9.96947607e-01, ...,
        4.36115802e-04, 4.35954525e-04, 4.36051651e-04],
       [9.96946183e-01, 4.05246028e-04, 6.22374179e-04, ...,
        4.05066306e-04, 4.05272045e-04, 4.05302216e-04]])

In [62]:
movie_df['Topic'] = topic_result.argmax(axis = 1)

In [63]:
movie_df

,label,review,scores,compound,compound_score,Topic
0,neg,how do films like mouse hunt get into theatres...,"{'neg': 0.121, 'neu': 0.778, 'pos': 0.101, 'co...",-0.9125,neg,7
1,neg,some talented actresses are blessed with a dem...,"{'neg': 0.12, 'neu': 0.775, 'pos': 0.105, 'com...",-0.8618,neg,0
2,pos,this has been an extraordinary year for austra...,"{'neg': 0.068, 'neu': 0.781, 'pos': 0.15, 'com...",0.9951,pos,0
3,pos,according to hollywood movies made in last few...,"{'neg': 0.071, 'neu': 0.782, 'pos': 0.147, 'co...",0.9972,pos,1
4,neg,my first press screening of 1998 and already i...,"{'neg': 0.091, 'neu': 0.817, 'pos': 0.093, 'co...",-0.2484,neg,0
...,...,...,...,...,...,...
1995,pos,"i like movies with albert brooks , and i reall...","{'neg': 0.073, 'neu': 0.763, 'pos': 0.164, 'co...",0.9991,pos,0
1996,pos,it might surprise some to know that joel and e...,"{'neg': 0.238, 'neu': 0.688, 'pos': 0.074, 'co...",-0.9993,neg,0
1997,pos,the verdict : spine-chilling drama from horror...,"{'neg': 0.15, 'neu': 0.702, 'pos': 0.147, 'com...",-0.5966,neg,0
1998,pos,i want to correct what i wrote in a former ret...,"{'neg': 0.131, 'neu': 0.71, 'pos': 0.16, 'comp...",0.9387,pos,2


In [69]:
def open_file(path):
  with open(path) as f:
    text = f.read()
    return text

In [70]:
# print(open_file('/content/melville-moby_dick.txt'))

In [71]:
nlp2.max_length = 1198623

In [67]:
def tokns(docmnt):
  return [token.text.lower() for token in nlp2(docmnt)]

In [73]:
d = open_file('/content/moby_dick_four_chapters.txt')

In [74]:
tokens = tokns(d)

/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [75]:
len(tokens)

14480

In [76]:
train_len = 25+1
text_sequences = []

for i in range(train_len, len(tokens)):
    seq = tokens[ i - train_len : i ]
    text_sequences.append(seq)

In [77]:
# arr = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
# res = []
# for i in range(len(arr)):
#   seq = arr[ i - 5 : i]
#   res.append(seq)
# res

In [78]:
print(' '.join(text_sequences[0]))

call me ishmael .   some years ago -- never mind how long 
 precisely -- having little or no money in my purse , and


In [79]:
print(' '.join(text_sequences[20]))

money in my purse , and nothing 
 particular to interest me on shore , i thought i would sail about a 
 little and see


In [80]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')

In [81]:
tokenizer.fit_on_texts(text_sequences)

In [82]:
sequence = tokenizer.texts_to_sequences(text_sequences)

In [83]:
# tokenizer.index_word

In [84]:
# tokenizer.word_counts

In [85]:
sequences = np.array(sequence)

In [86]:
vocabulary_size = len(tokenizer.word_counts)
vocabulary_size

2734

In [87]:
# sequence

In [88]:
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

In [89]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, 25, input_length=seq_len))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(150))
    model.add(Dense(150, activation='relu'))

    model.add(Dense(vocabulary_size, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.summary()

    return model

In [90]:
from keras.utils import to_categorical

In [91]:
X = sequences[:,:-1]
y = sequences[:,-1]

In [92]:
y = to_categorical(y, num_classes=vocabulary_size+1)

In [93]:
seq_len = X.shape[1]

In [94]:
model = create_model(vocabulary_size+1, seq_len)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 25)            68375     
                                                                 
 lstm (LSTM)                 (None, 25, 150)           105600    
                                                                 
 lstm_1 (LSTM)               (None, 150)               180600    
                                                                 
 dense (Dense)               (None, 150)               22650     
                                                                 
 dense_1 (Dense)             (None, 2735)              412985    
                                                                 
Total params: 790210 (3.01 MB)
Trainable params: 790210 (3.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [95]:
model.fit(X, y, batch_size=128, epochs=160,verbose=1)

Epoch 1/160
113/113 [==============================] - 16s 102ms/step - loss: 6.3814 - accuracy: 0.0545
Epoch 2/160
113/113 [==============================] - 5s 43ms/step - loss: 5.9514 - accuracy: 0.0619
Epoch 3/160
113/113 [==============================] - 4s 31ms/step - loss: 5.8890 - accuracy: 0.0628
Epoch 4/160
113/113 [==============================] - 2s 19ms/step - loss: 5.7424 - accuracy: 0.0809
Epoch 5/160
113/113 [==============================] - 2s 17ms/step - loss: 5.6037 - accuracy: 0.1077
Epoch 6/160
113/113 [==============================] - 1s 12ms/step - loss: 5.4599 - accuracy: 0.1191
Epoch 7/160
113/113 [==============================] - 1s 12ms/step - loss: 5.3448 - accuracy: 0.1218
Epoch 8/160
113/113 [==============================] - 1s 12ms/step - loss: 5.2447 - accuracy: 0.1279
Epoch 9/160
113/113 [==============================] - 2s 15ms/step - loss: 5.1678 - accuracy: 0.1329
Epoch 10/160
113/113 [==============================] - 1s 12ms/step - loss: 5.0

In [96]:
from pickle import dump,load

In [97]:
model.save('mymodel.h5')
dump(tokenizer, open('mymodel', 'wb'))

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [98]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [99]:
def generated_text(model, tokenizer, seq_len, seed_text, num_gen_words):

    # Final Output
    output_text = []

    # Initial Seed Sequence
    input_text = seed_text

    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

        # Get predicted probabilities for the next word
        pred_probabilities = model.predict(pad_encoded, verbose=0)[0]

        # Get the index of the word with the highest probability
        pred_word_ind = np.argmax(pred_probabilities)

        # Retrieve the word corresponding to the index
        pred_word = tokenizer.index_word.get(pred_word_ind)  # Use get method to handle out-of-vocabulary indices

        # Append the predicted word to the input text
        input_text += ' ' + pred_word
        output_text.append(pred_word)

    # Make it look like a sentence
    return ' '.join(output_text)


In [100]:
import random
random_pick = random.randint(0,len(text_sequences))

In [101]:
random_seed_text = text_sequences[random_pick]

In [102]:
seed_text = ' '.join(random_seed_text)
print(seed_text)

chest , arms , and hands .   he then donned his 
 waistcoat , and taking up a piece of hard soap on the wash


In [103]:
generated_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=20)

'- - - - - - - - - - - - - - - - - - - -'

In [104]:
from keras.models import load_model

In [105]:
# model = load_model('/content/epochBIG.h5')

In [106]:
# tokenizer = load(open('/content/epochBIG','rb'))

### Beam search

In [107]:
def beam_search(model, tokenizer, seq_len, seed_text, num_gen_words, k=3):
    sequences = [[tokenizer.texts_to_sequences([seed_text])[0], 0.0]]

    while len(sequences[0][0]) < num_gen_words:
        all_candidates = []
        for seq, score in sequences:
            pad_encoded = pad_sequences([seq], maxlen=seq_len, truncating='pre')
            pred_probabilities = model.predict(pad_encoded, verbose=0)[0]
            top_k_preds = np.argsort(pred_probabilities)[-k:]  # Top k predictions

            for pred in top_k_preds:
                candidate_seq = [seq + [pred], score - np.log(pred_probabilities[pred])]
                all_candidates.append(candidate_seq)

        # Sort candidates by score
        sequences = sorted(all_candidates, key=lambda tup: tup[1])[:k]

    # Retrieve the best sequence
    best_sequence = sequences[0][0]
    output_text = [tokenizer.index_word.get(idx, '[UNK]') for idx in best_sequence]

    return ' '.join(output_text)

In [108]:
generated_text = beam_search(model, tokenizer, seq_len, seed_text, num_gen_words=40)
print(generated_text)

chest arms and hands he then donned his waistcoat and taking up a piece of hard soap on the wash - stand 
 centre table , dipped it into water and commenced lathering his face . 
 i was watching


#chatbots



In [109]:
import pickle
with open('/content/train_qa.txt','rb') as f:
  train_data = pickle.load(f)

In [110]:
with open('/content/test_qa.txt','rb') as f:
  test_data = pickle.load(f)

In [111]:
all_data  = train_data + test_data

In [112]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [113]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [114]:
' '.join(train_data[0][2])

'n o'

In [115]:
df  = pd.DataFrame(all_data,columns=['Story','Question','Answer'])
df

,Story,Question,Answer
0,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Sandra, in, the, hallway, ?]",no
1,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Daniel, in, the, bathroom, ?]",no
2,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Daniel, in, the, office, ?]",no
3,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Daniel, in, the, bedroom, ?]",yes
4,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Daniel, in, the, bedroom, ?]",yes
...,...,...,...
10995,"[Mary, moved, to, the, kitchen, ., Mary, trave...","[Is, Mary, in, the, bedroom, ?]",no
10996,"[Mary, moved, to, the, kitchen, ., Mary, trave...","[Is, Sandra, in, the, kitchen, ?]",no
10997,"[Mary, moved, to, the, kitchen, ., Mary, trave...","[Is, Mary, in, the, bedroom, ?]",no
10998,"[Mary, moved, to, the, kitchen, ., Mary, trave...","[Is, Sandra, in, the, garden, ?]",yes


In [116]:
vocab = set()
for story,question,answer in all_data:
  vocab = vocab.union(set(story))
  vocab = vocab.union(set(question))


In [117]:
vocab.add('yes')
vocab.add('no')
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [118]:
vocab_len = len(vocab) + 1
# for i in range(len(all_data)):
#   for data in all_data[i]:
#     print(' '.join(data))

In [119]:
stories_len = [len(data[0]) for data in all_data]
question_len = [len(data[1]) for data in all_data]

In [120]:
maxlen_stories = max(stories_len)
maxlen_stories

156

In [121]:
maxlen_question = max(question_len)
maxlen_question

6

In [122]:
tokenizer = Tokenizer(filters = [])

In [123]:
tokenizer.fit_on_texts(vocab)

In [124]:
tokenizer.word_index['yes']

36

In [125]:
tokenizer.word_index['no']

10

In [126]:
tokenizer.word_index

{'dropped': 1,
 'bathroom': 2,
 'left': 3,
 'got': 4,
 'to': 5,
 'the': 6,
 'football': 7,
 'took': 8,
 'office': 9,
 'no': 10,
 '.': 11,
 'back': 12,
 'there': 13,
 'in': 14,
 'hallway': 15,
 'bedroom': 16,
 'daniel': 17,
 'discarded': 18,
 'milk': 19,
 'sandra': 20,
 '?': 21,
 'apple': 22,
 'john': 23,
 'put': 24,
 'down': 25,
 'kitchen': 26,
 'garden': 27,
 'went': 28,
 'moved': 29,
 'picked': 30,
 'grabbed': 31,
 'travelled': 32,
 'up': 33,
 'is': 34,
 'journeyed': 35,
 'yes': 36,
 'mary': 37}

In [127]:
train_story_text = []
train_question_text = []
train_answer_text = []
for s,q,a in train_data:
  train_story_text.append(s)
  train_question_text.append(q)
  train_answer_text.append(q)

In [128]:
# train_story_text

In [129]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)
# train_story_seq = np.array(train_story_seq)
# train_story_seq

In [130]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=maxlen_stories,max_question_len=maxlen_question):
    '''
    OUTPUT:
    Vectorizes the stories,questions, and answers into padded sequences. We first loop for every story, query , and
    answer in the data. Then we convert the raw words to an word index value. Then we append each set to their appropriate
    output list. Then once we have converted the words to numbers, we pad the sequences so they are all of equal length.
    Returns this in the form of a tuple (X,Xq,Y) (padded based on max lengths)
    '''
    # X = STORIES
    X = []
    # Xq = QUERY/QUESTION
    Xq = []
    # Y = CORRECT ANSWER
    Y = []


    for story, query, answer in data:

        # Grab the word index for every word in story
        x = [word_index[word.lower()] for word in story]
        xq = [word_index[word.lower()] for word in query]
        y = np.zeros(len(word_index) + 1)
        y[word_index[answer]] = 1

        X.append(x)
        Xq.append(xq)
        Y.append(y)

    return (pad_sequences(X, maxlen=max_story_len),pad_sequences(Xq, maxlen=max_question_len), np.array(Y))

In [131]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)
inputs_test, queries_test, answers_test = vectorize_stories(test_data)

In [132]:
import keras.layers
from keras.models import Sequential, Model
from keras.layers import Embedding,Input, Activation, Dense, Permute, Dropout,add, dot, concatenate,LSTM

In [133]:
input_sequence = Input((maxlen_stories,))
question_sequence = Input((maxlen_question,))
question_sequence

<KerasTensor: shape=(None, 6) dtype=float32 (created by layer 'input_2')>

### input encoder M

In [134]:
# Input gets embedded to a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_len,output_dim=64))
input_encoder_m.add(Dropout(0.5))

# This encoder will output:
# (samples, story_maxlen, embedding_dim)

### input encoder C

In [135]:
# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_len,output_dim=maxlen_question))
input_encoder_c.add(Dropout(0.5))
# output: (samples, story_maxlen, query_maxlen)

### question encoder


In [136]:
# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_len,
                               output_dim=64,
                               input_length=maxlen_question))
question_encoder.add(Dropout(0.5))
# output: (samples, query_maxlen, embedding_dim)

In [137]:
# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question_sequence)

In [138]:
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

In [139]:
# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

In [140]:
# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])
answer

<KerasTensor: shape=(None, 6, 220) dtype=float32 (created by layer 'concatenate')>

In [141]:
# Reduce with RNN (LSTM)
answer = LSTM(32)(answer)  # (samples, 32)
# Regularization with Dropout
answer = Dropout(0.5)(answer)
answer = Dense(vocab_len)(answer)  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

In [142]:
model = Model([input_sequence, question_sequence], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [143]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 156)]                0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 6)]                  0         []                            
                                                                                                  
 sequential_1 (Sequential)   (None, None, 64)             2432      ['input_1[0][0]']             
                                                                                                  
 sequential_3 (Sequential)   (None, 6, 64)                2432      ['input_2[0][0]']             
                                                                                              

In [144]:
# train
history = model.fit([inputs_train, queries_train], answers_train,batch_size=32,epochs=100,validation_data=([inputs_test, queries_test], answers_test))

Epoch 1/100
313/313 [==============================] - 8s 12ms/step - loss: 0.9514 - accuracy: 0.4927 - val_loss: 0.6996 - val_accuracy: 0.4970
Epoch 2/100
313/313 [==============================] - 2s 6ms/step - loss: 0.7115 - accuracy: 0.4915 - val_loss: 0.6935 - val_accuracy: 0.5030
Epoch 3/100
313/313 [==============================] - 2s 6ms/step - loss: 0.6994 - accuracy: 0.4899 - val_loss: 0.6935 - val_accuracy: 0.5030
Epoch 4/100
313/313 [==============================] - 2s 6ms/step - loss: 0.6964 - accuracy: 0.5031 - val_loss: 0.6932 - val_accuracy: 0.5030
Epoch 5/100
313/313 [==============================] - 2s 6ms/step - loss: 0.6958 - accuracy: 0.5030 - val_loss: 0.6970 - val_accuracy: 0.5030
Epoch 6/100
313/313 [==============================] - 2s 6ms/step - loss: 0.6954 - accuracy: 0.4977 - val_loss: 0.6951 - val_accuracy: 0.4970
Epoch 7/100
313/313 [==============================] - 3s 8ms/step - loss: 0.6953 - accuracy: 0.5008 - val_loss: 0.6932 - val_accuracy: 0.503

In [145]:
filename = 'chatbot.h5'
model.save(filename)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [146]:
model.load_weights(filename)
pred_results = model.predict(([inputs_test, queries_test]))

32/32 [==============================] - 0s 2ms/step


In [147]:
story =' '.join(word for word in test_data[1][0])
print(story)

Mary got the milk there . John moved to the bedroom . Mary discarded the milk . John went to the garden .


In [148]:
query = ' '.join(word for word in test_data[1][1])
print(query)

Is John in the kitchen ?


In [149]:
print("Answer is:",test_data[1][2])

Answer is: no


In [150]:
#Generate prediction from model
val_max = np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer is: ", k)
print("Probability of certainty was: ", pred_results[0][val_max])

Predicted answer is:  no
Probability of certainty was:  0.9801676


In [151]:
# Note the whitespace of the periods
my_story = "John left the kitchen . Sandra dropped the football in the garden ."
my_story.split()

['John',
 'left',
 'the',
 'kitchen',
 '.',
 'Sandra',
 'dropped',
 'the',
 'football',
 'in',
 'the',
 'garden',
 '.']

In [152]:
my_question = "Is the football in the garden ?"
my_question.split()

['Is', 'the', 'football', 'in', 'the', 'garden', '?']

In [153]:
mydata = [(my_story.split(),my_question.split(),'yes')]

In [154]:
my_story,my_ques,my_ans = vectorize_stories(mydata)

In [155]:
pred_results = model.predict(([ my_story, my_ques]))
pred_results

1/1 [==============================] - 0s 41ms/step


array([[2.6645008e-08, 2.7434293e-08, 2.3168976e-08, 2.5048005e-08,
        3.0226747e-08, 2.3394470e-08, 2.8759285e-08, 2.8762082e-08,
        2.7334844e-08, 2.8390959e-08, 2.3132144e-02, 2.1616055e-08,
        2.7588198e-08, 2.1404272e-08, 2.5494286e-08, 2.4696339e-08,
        2.6864491e-08, 2.9621535e-08, 2.5999746e-08, 2.3783240e-08,
        2.5914046e-08, 2.4919533e-08, 2.4195490e-08, 2.4068683e-08,
        2.6952048e-08, 2.5983335e-08, 2.3464542e-08, 2.3430506e-08,
        2.9014215e-08, 2.3250564e-08, 2.6113156e-08, 2.5577378e-08,
        2.4648104e-08, 3.1723552e-08, 3.1309860e-08, 2.1627891e-08,
        9.7686702e-01, 2.2923652e-08]], dtype=float32)

In [156]:
#Generate prediction from model
val_max = np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer is: ", k)
print("Probability of certainty was: ", pred_results[0][val_max])

Predicted answer is:  yes
Probability of certainty was:  0.976867
